### Importing

In [1]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

air_canada = pd.read_csv('../datasets/new_branded_data.csv')

# seperate dataset into training and predicting
air_canada_to_predict = air_canada[air_canada['choice'] != air_canada['choice']]
air_canada = air_canada[air_canada['choice'] == air_canada['choice']]

X, y = air_canada.drop(['Unnamed: 0', 'id', 'ticket_id', 'choice'], axis=1), air_canada[['choice']]

y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

/tmp/ipykernel_367504/1501615036.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })


In [2]:
X.columns

Index(['od', 'flight_departure_datetime', 'purchase_datetime', 'time_diff',
       'trip_type', 'branded_fare', 'number_of_pax', 'ADVS_price',
       'PREF_price', 'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory',
       'PREF_inventory', 'pref_inv_full', 'advs_ratio', 'pref_ratio',
       'total_cap'],
      dtype='object')

### Preprocessing

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [4]:
num_pipeline = make_pipeline(
  StandardScaler()
)

cat_pipeline = make_pipeline(
  OneHotEncoder()
)
preprocessing = ColumnTransformer([
    ('cat', cat_pipeline, ['od', 'trip_type', 'branded_fare', 'total_cap']),
    ('num', num_pipeline, ['time_diff', 'number_of_pax', 'ADVS_price', 'PREF_price',
                           'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory',
                           'pref_inv_full', 'advs_ratio', 'pref_ratio'])
  ], remainder='drop')

In [5]:
X_train_prep = preprocessing.fit_transform(X_train)
X_test_prep = preprocessing.transform(X_test)

### Training

In [6]:
from lightgbm import LGBMClassifier

params = {
    'objective': 'multiclass',
    'force_row_wise': True,
    'num_class': 3,
}

bst = LGBMClassifier(**params)

bst.fit(X_train_prep, y_train['choice'])

y_pred = bst.predict(X_test_prep)

print("Accuracy:", accuracy_score(y_pred, y_test))
print("F1 Score:", f1_score(y_pred, y_test, average='weighted'))

[LightGBM] [Info] Total Bins 877
[LightGBM] [Info] Number of data points in the train set: 173248, number of used features: 30
[LightGBM] [Info] Start training from score -0.386339
[LightGBM] [Info] Start training from score -1.919581
[LightGBM] [Info] Start training from score -1.749900
Accuracy: 0.7464935064935065
F1 Score: 0.7863403247005998


### Predicting

In [8]:
X_to_predict, y_to_predict = air_canada_to_predict.drop(['id', 'ticket_id', 'choice'], axis=1), air_canada_to_predict[['id']]
X_to_predict_prep = preprocessing.transform(X_to_predict)
y_to_predict['choice'] = bst.predict(X_to_predict_prep)

/tmp/ipykernel_367504/1900845532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = bst.predict(X_to_predict_prep)


In [9]:
y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})

/tmp/ipykernel_367504/1816271922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})


In [10]:
y_final = pd.concat([y_to_predict, air_canada])[['id', 'choice']]

### Saving

In [12]:
pd.DataFrame.to_csv(y_final, '../predictions/pred9-lgb.csv', index=False)